# Métodos Iterativos #

### Critérios de parada ###

Antes de implementarmos os métodos iterativos, temos que implementar um método para o critério de parada. No nosso caso, usaremos a _norma infinita_. A norma infinita é dada por:

$$\lVert v \rVert_{\infty} = \max_{1\leq i\leq n}|v_i|$$ 

E a distância entre duas soluções iteradas $x^k$ e $x^{k-1}$ pode ser dada por:

$$ \frac{\lVert x^k - x^{k-1} \rVert_{\infty}}{\lVert x^k\rVert_{\infty}} = \frac{\max\limits_{1\leq i\leq n}|x_i^{k} - x_i^{k-1}|}{\max\limits_{1\leq i\leq n}|x_i^k|} \leq \epsilon$$

Faça uma função para retornar a distância usando a norma infinita de dois vetores:

In [ ]:
import numpy as np

In [ ]:
x0 = np.array([5.7, 2.5 , -0.8])
x1 = np.array([4.79, 0.975, -2.44])

#tem outra forma de implementar a normaInf
def normaInf(v1, v2):
    norma = 0
    v1 = abs(v1)
    v2 = abs(v2)
    x = np.zeros(len(v1))
    for i in range(len(v1)):
        x[i] = abs(v2[i] - v1[i])
    norma = max(x)/max(v2)
    return norma

#essa q é usando a magica do python
def normaInfversion2(v1,v2):
    norma =0
    for i in range(len(v1)):
        norma = np.linalg.norm(v2 - v1, np.inf)/np.linalg.norm(v2, np.inf)
    return norma

#normaInfversion2(x0,x1)

normaInf(x0, x1) 
#mas eu usei a versao 1 msm

## Jacobi ##

Para resolver o método de Jacobi, se deve decompor a matriz $A$ em três matrizes compostas dos seus elementos, $D$ (Diagonal), $E$ (triangular inferior) e $F$ (triangular superior).


De posse destes elementos, deve-se utilizar esta fórmula de iteração:

$$ x^{k+1}=-D^{-1}(E+F)x+D^{-1}b $$

Lembrando que para D ter inversa, nenhum elemento da diagonal principal de A pode ser nulo.

Desta forma, eu posso utilizar a forma matricial acima para computar as respostas. Implemente o método de Jacobi matricial abaixo:

In [ ]:
def jacobiM(A,b, niter = 1000, minimo=0.000001):
    A = A.astype('double')  
    b = b.astype('double')
    Di = np.diag(np.diag(1/A))
    j = np.zeros(len(A))
    x = np.copy(b)
    EF = A -np.diag(np.diag(A))
    n = 0
    xk = np.zeros(len(b))
    #decompondo
    j = np.matmul(-Di,EF) # gerar a matriz j
    x = np.matmul(Di,b)
    # norma infinita e o k maximo de iteraçoes
    # criterio de parada para o minimo e quando a distancia dos dois vetores for menor q o minimo
    while( (n < niter) or (normaInf(xk,x) > minimo)): 
        xk = x
        x = np.matmul(j,x) + np.matmul(Di,b)  # aq e a formula x^k+1 = j*x^k + c
        n +=1
    return x

In [ ]:
a = np.array([[10,3,-2], 
              [2,8,-1],
              [1,1,5]])
b = np.array([57,20, -4])
jacobiM(a,b)

Outra forma de implementar Jacobi é através da fórmula escalar:

$$x_i^{k+1}=\frac{1}{a_{ii}} - \left( \sum\limits_{\substack{j=1 \\ i\neq j}}^{n}{a_{ij}x_j^k+b_i} \right)$$

Implemente a forma escalar do jacobi:

In [ ]:
def jacobiE(A,b,niter=1000,minimo = 0.000001):   
    A = A.astype('double')  
    b = b.astype('double')
    xk = np.zeros(len(b))
    n=np.shape(A)[0]  
    x = np.copy(b)  
    it = 0 
    while (it < niter or normaInf(xk,x) > minimo):
        it = it+1 
        for i in np.arange(n):  
            x[i] = b[i]  
            for j in np.concatenate((np.arange(0,i),np.arange(i+1,n))):  
                x[i] -= A[i,j]*xk[j]  
            x[i] /= A[i,i] 
        xk = np.copy(x)
    return x


In [ ]:

a = np.array([[10,3,-2], 
              [2,8,-1],
              [1,1,5]])
b = np.array([57,20, -4])

jacobiE(a,b)

Para o método de Jacobi (e o Gauss-Seidel) convergirem para um resultado correto, elas tem que ter condições de convergência:

- Condição suficiente: Elementos da diagonal principal estritamente dominantes.

$$ |a_{ii}|> \sum\limits_{\substack{j=1 \\ j\neq i}}^{n}{|a_{ij}|,i=1,2,...,n}$$

Esta condição se verdadeira garante a convergência. Contudo, mesmo se ela for falsa o sistema ainda pode convergir.

- Condição necessária: $\rho(M)<1$, sendo $\rho(M)$ o raio espectral, o maior autovalor em módulo.

Faça uma função que verifique se um sistema com uma matriz de coeficientes A pode ser solucionado via jacobi, testando primeiro a condição suficiente e, em caso negativo, testando a condição necessária:

In [ ]:
def verificaDominante(A):
    v = A.diagonal()
    for i in range(len(A)):
        for j in range(len(A)):
            if(i != j):
                if(v[j] < A[i,j]):
                    return False
    return True

def verificaRaioEspectral(A):
    (x,_) = np.linalg.eig(A)
    if (max(x) < 1):
        return True
    else:
        return False
    

def podeJacobi(A):
    t = verificaDominante(A)
    v = verificaRaioEspectral(A)
    if(t == True or v == True):
        return True
    else:
        return False

M3 = np.array([[9,-6,3],
               [-6,29,-7],
               [3,-7,18]])
b3 = np.array([-3,-8,33])

verificaDominante(M3)


## Gauss-Seidel ##

Função de iteração:

$$ x^{k+1}=D^{-1}(-Ex^{k+1}-Fx^{k}+b)$$

Lembrando que multiplicar por $D^{-1}$ é o mesmo que dividir cada linha pelo elemento da diagonal principal desta linha em A

A forma mais simples de fazer o Gauss Seidel matricialmente é linha a linha (calculando a formula acima pra cada linha da solução X)

Implemente Gauss-Seidel na forma Matricial e na forma escalar (acessando os elementos individualmente das matrizes)

In [ ]:
def gaussM(A,b,niter=1000,minimo = 0.000001):
    n = 0
    Di = np.diag(np.diag(1/A))
    E = np.tril(A, k=-1)
    F = np.triu(A, k = 1)
    xk = np.zeros(len(b))
    r = np.zeros(len(A))
    x = np.matmul(Di,b)
    Fx = np.matmul(F,x)
    while(n < niter or normaInf(xk,x) > minimo):
        xk = x
        Ex = np.matmul(E,x)
        t = - Ex - Fx + b
        x = np.matmul(Di,t)
        n += 1
    return x


In [ ]:
a = np.array([[10,3,-2], 
              [2,8,-1],
              [1,1,5]])
b = np.array([57,20, -4])

gaussM(a,b)

In [ ]:
def gaussE(A,b,niter=1000,minimo = 0.000001):  
    #preliminares  
    A = A.astype('double')  
    b = b.astype('double')
    xk = np.zeros(len(b))
    n=np.shape(A)[0]  
    x = np.copy(b)  
    it = 0 
    while (it < niter or normaInf(xk, x) > minimo):  
        it = it+1
        xk = x  
        for i in np.arange(n):  
            x[i] = b[i]  
            for j in np.concatenate((np.arange(0,i),np.arange(i+1,n))):  
                x[i] -= A[i,j]*x[j]  
            x[i] /= A[i,i]  
    return n
         

In [ ]:
a = np.array([[10,3,-2], 
              [2,8,-1],
              [1,1,5]])
b = np.array([57,20, -4])

gaussE(a,b)

# Implementação dos metodos exatos:##
Tringular Superior:

In [ ]:
def resolveTS(A,b): #A Triangular Superior
    x= np.zeros(len(A))
    for i in range(len(A)-1, -1,-1):
        x[i]=(b[i]-(A[i,i+1:]*x[i+1:]).sum())/A[i,i]
    return x

Triangular Inferior:

In [ ]:
def resolveTI(A,b): #A Triangular Inferior
    m = len(A)
    n = len(A[0])
    k=0
    x = np.zeros(n)
    for i in range(0,m):
        k=0
        for j in range(0,n-1):
            k = A[i][j] * x[j]
        x[i] = (b[i]-k)/A[i][i]
        
    return x

## Eliminação Gaussiana

In [ ]:
def eliminacaoGaussianaSimples(A,b):
    pivot = 0
    m = 0.0
    x = np.zeros(len(A))
    for i in range(0,len(A)-1):
        pivot = A[i,i]
        for j in range(i+1,len(A)):
            if (pivot == 0):
                m = 0
            else:
                m = A[j,i]/pivot
            A[j] =  A[j] - (m*A[i])
            b[j] = b[j] - (m*b[i])       
    x = resolveTS(A,b)
    return x

## Decomposição LU ##


In [ ]:
# decompor A para:
#deixar o triangulo inferior nulo para isso usar a funcao 

def decompoeLU(A):  
    U = np.copy(A)  
    n = np.shape(U)[0]  
    L = np.eye(n)  
    for j in np.arange(n-1):  
        for i in np.arange(j+1,n):  
            L[i,j] = U[i,j]/U[j,j]  
            for k in np.arange(j+1,n):  
                U[i,k] = U[i,k] - L[i,j]*U[j,k]  
            U[i,j] = 0  
    return (L, U)


#funciona
def resolveLU(L,U,b):
    t = resolveTI(L,b)
    x = resolveTS(U,t)
    return x

#### Pivotação Parcial ###

In [ ]:
def verificaPivot(A):# escolher a maior linha em modulo
    for i in range(0,len(A)):
        if np.linalg.det(A[:i+1,:i+1]) != 0:
            return 0
    return 1

def LUparcial(A):
    P = np.eye(len(A))
    L = np.eye(len(A))
    i = 0
    for i in range(0,len(A)-1):
        im = np.argmax(abs(A[i:,i])) + i
        pivo = A[im][i]
        for j in range(i,len(A)):        
            m = A[j][i] / pivo
            if j != im: 
                A[j] = A[j] - A[im] * m
                if j < im: 
                    L[j+1][i] = m
                else: 
                    L[j][i] = m
    
        A[[i,im]]=A[[im,i]]
        P[[i,im]]=P[[im,i]]    
    U = A
    return L,U,P

def resolveLUpar(L,U,P,b):
    y = resolveTI(L,np.dot(P,b))
    x = resolveTS(U, y)
    return x


## Cholesky ##

In [ ]:
#sim, funciona
def simetrica(A):
    At = A[:,:].T
    x = np.equal(A, At)
    if(np.all(x == True)): #verificar se a A e igual a transposta de A (Simetria)
        return True

def diagonalPositiva(A):
    if(np.all(A.diagonal() > 0)): #verificar se a diagonal principal e positiva
        return True

def autoValPos(A):
    (a,_) = np.linalg.eig(A)
    for i in range(len(A)): # autovalores positivos
        if(a[i] < 0):
            return False
        else:
            return True
def subMatDetPos(A):
     for j in range(len(A)):
        t = np.linalg.det(A[0:j,0:j]) # determinantes das submatrizes positivas
        if(t < 0):
            return False
        else:
            return True      
def verificaCholesky(A):
    x = simetrica(A)
    y = diagonalPositiva(A)
    t = autoValPos(A)
    h = subMatDetPos(A)
    if ((x == True) and (y==True) and (t==True) and (h==True)):
        return True
    else:
        return False
      

def geraCholesky(A):
    L = [[0.0] * len(A) for _ in range(len(A))]
    for i, (Ai, Li) in enumerate(zip(A, L)):
        for j, Lj in enumerate(L[:i+1]):
            s = sum(Li[k] * Lj[k] for k in range(j))
            Li[j] = np.sqrt(Ai[i] - s) if (i == j) else \
                      (1.0 / Lj[j] * (Ai[j] - s))
    L = np.array(L)
    return L

def resolveCholesky(A,b):
    a = verificaCholesky(A)
    if(a == True):
        L = geraCholesky(A) #gera L
        x = resolveLU(L,L.T,b) # resolve L e a transposta de L
        return x
    else:
        return -1

def detCholesky(A):
    L = geraCholesky(A)
    detL = np.linalg.det(L)
    detLT = np.linalg.det(L.T)
    det = detL*detLT
    return det

#### Exercicios ####

1 - Tente resolver todos os sistemas (de M1 a M5) com Jacobi e Gauss-Seidel, tanto na forma matricial quanto na escalar e, nos casos em que o resultado _divergir_, verifique a condição de convergência dos métodos.



In [ ]:
#antes de testar para cada uma das matrizes
#funçao pra verificar se a matriz pode ser resolvida 

# Jacobi escalar
def TestarFazerJacobiE(M, b):
    x = 0
    if(podeJacobi(M) == True):
        print("Pode:")
        x = jacobiE(M, b)
        return x
    else:
        print("Não. Ele falhou em uma das condições de convergência ")
        print("Falhou na condiçao suficiente da diagonal dominante?", verificaDominante(M))
        print("Falhou na condiçao necessaria do raio espectral? ", verificaRaioEspectral(M))
        
        
#Jacobi matricial
def TestarFazerJacobiM(M, b):
    x = 0
    if(podeJacobi(M) == True):
        print("Pode:")
        x = jacobiM(M, b)
        return x
    else:
        print("Não. Ele falhou em uma das condições de convergência ")
        print("Falhou na condiçao suficiente da diagonal dominante?", verificaDominante(M))
        print("Falhou na condiçao necessaria do raio espectral? ", verificaRaioEspectral(M))
        
#Gauss-Seidel escalar
def TestarFazerGauss_SeidelE(M, b):
    x = 0
    if(podeJacobi(M) == True):
        print("Pode:")
        x = gaussE(M, b)
        return x
    else:
        print("Não. Ele falhou em uma das condições de convergência ")
        print("Falhou na condiçao suficiente da diagonal dominante?", verificaDominante(M))
        print("Falhou na condiçao necessaria do raio espectral? ", verificaRaioEspectral(M))

#Gauss-Seidel matricial
def TestarFazerGauss_SeidelM(M, b):
    x = 0
    if(podeJacobi(M) == True):
        print("Pode:")
        x = gaussM(M, b)
        return x
    else:
        print("Não. Ele falhou em uma das condições de convergência ")
        print("Falhou na condiçao suficiente da diagonal dominante?", verificaDominante(M))
        print("Falhou na condiçao necessaria do raio espectral? ", verificaRaioEspectral(M))


#### Para a matriz M1: #### 

In [ ]:
M1 = np.array([[1,-3,5,6], 
               [-8,4,-1,0],
               [3,2,-2,7],
               [1,2,5,-4]])
b1 = np.array([17,29,-11,7])


print("Metodo de Jacobi pela forma escalar:\n")

print("A matriz pode resolvida?\n")
TestarFazerJacobiE(M1, b1)

In [ ]:
M1 = np.array([[1,-3,5,6], 
               [-8,4,-1,0],
               [3,2,-2,7],
               [1,2,5,-4]])
b1 = np.array([17,29,-11,7])
print("Metodo de Jacobi pela forma matricial:\n")

print("A matriz pode resolvida?\n")
TestarFazerJacobiM(M1, b1)

In [ ]:
M1 = np.array([[1,-3,5,6], 
               [-8,4,-1,0],
               [3,2,-2,7],
               [1,2,5,-4]])
b1 = np.array([17,29,-11,7])

print("Metodo de Gauss-Seidel pela forma escalar:\n")

TestarFazerGauss_SeidelE(M1, b1)

In [ ]:
M1 = np.array([[1,-3,5,6], 
               [-8,4,-1,0],
               [3,2,-2,7],
               [1,2,5,-4]])
b1 = np.array([17,29,-11,7])
print("Metodo de Gauss-Seidel pela forma matricial:\n")

TestarFazerGauss_SeidelM(M1, b1)

#### Para a matriz M2: ####

In [ ]:
M2 = np.array([[-2,3,1,5],
               [5,1,-1,0],
               [1,6,3,-1],
               [4,5,2,8]])
b2 = np.array([2,-1,0,6])

print("Metodo de Jacobi pela forma escalar:\n")

print("A matriz pode resolvida?\n")
TestarFazerJacobiE(M2, b2)

In [ ]:
M2 = np.array([[-2,3,1,5],
               [5,1,-1,0],
               [1,6,3,-1],
               [4,5,2,8]])
b2 = np.array([2,-1,0,6])

print("Metodo de Jacobi pela forma matricial:\n")
print("A matriz pode resolvida?\n")
TestarFazerJacobiM(M2, b2)

In [ ]:
M2 = np.array([[-2,3,1,5],
               [5,1,-1,0],
               [1,6,3,-1],
               [4,5,2,8]])
b2 = np.array([2,-1,0,6])
print("Metodo de Gauss-Seidel pela forma escalar:")

TestarFazerGauss_SeidelE(M2, b2)

In [ ]:
M2 = np.array([[-2,3,1,5],
               [5,1,-1,0],
               [1,6,3,-1],
               [4,5,2,8]])
b2 = np.array([2,-1,0,6])
print("Metodo de Gauss-Seidel pela forma matricial:")
print("A matriz pode resolvida?\n")
TestarFazerGauss_SeidelM(M2, b2)

#### Para a matriz M3: ####

In [ ]:
M3 = np.array([[9,-6,3],
               [-6,29,-7],
               [3,-7,18]])
b3 = np.array([-3,-8,33])

print("Metodo de Jacobi pela forma escalar:\n")

print("A matriz pode resolvida?\n")
TestarFazerJacobiE(M3, b3)

In [ ]:
M3 = np.array([[9,-6,3],
               [-6,29,-7],
               [3,-7,18]])
b3 = np.array([-3,-8,33])
print("Metodo de Jacobi pela forma matrical:")
TestarFazerJacobiM(M3, b3)

In [ ]:
M3 = np.array([[9,-6,3],
               [-6,29,-7],
               [3,-7,18]])
b3 = np.array([-3,-8,33])
print("Metodo de Gauss-Seidel pela forma escalar:")
TestarFazerGauss_SeidelE(M3, b3)

In [ ]:
M3 = np.array([[9,-6,3],
               [-6,29,-7],
               [3,-7,18]])
b3 = np.array([-3,-8,33])
print("Metodo de Gauss-Seidel pela forma Matricial:")
TestarFazerGauss_SeidelM(M3, b3)

#### Para a matriz M4: ####

In [ ]:
M4 = np.array([[4,-2,4,10],
               [-2,2,-1,-7],
               [4,-1,14,11],
               [10,-7,11,31]])
b4 = np.array([2,2,-1,-2])

print("Metodo de Jacobi pela forma escalar:\n")

print("A matriz pode resolvida?\n")
TestarFazerJacobiE(M4, b4)


In [ ]:
M4 = np.array([[4,-2,4,10],
               [-2,2,-1,-7],
               [4,-1,14,11],
               [10,-7,11,31]])
b4 = np.array([2,2,-1,-2])

print("Metodo de Jacobi pela forma matrical:")
print("A matriz pode resolvida?\n")
TestarFazerJacobiM(M1, b1)

In [ ]:
M4 = np.array([[4,-2,4,10],
               [-2,2,-1,-7],
               [4,-1,14,11],
               [10,-7,11,31]])
b4 = np.array([2,2,-1,-2])


print("Metodo de Gauss-Seidel pela forma escalar:")
print("A matriz pode resolvida?\n")
TestarFazerGauss_SeidelE(M4, b4)

In [ ]:
M4 = np.array([[4,-2,4,10],
               [-2,2,-1,-7],
               [4,-1,14,11],
               [10,-7,11,31]])
b4 = np.array([2,2,-1,-2])

print("Metodo de Gauss-Seidel pela forma Matricial:")
print("A matriz pode resolvida?\n")
TestarFazerGauss_SeidelM(M4, b4)

#### Para a matriz M5: ####

In [ ]:

M5 = np.array([[16,-4,4,12],
               [-4,2,-1,-7],
               [4,-1,26,13],
               [12,-7,13,25]])
b5 = np.array([2,2,-1,-2])


print("Metodo de Jacobi pela forma escalar:\n")

print("A matriz pode resolvida?\n")
TestarFazerJacobiE(M5, b5)

In [ ]:
M5 = np.array([[16,-4,4,12],
               [-4,2,-1,-7],
               [4,-1,26,13],
               [12,-7,13,25]])
b5 = np.array([2,2,-1,-2])

print("Metodo de Jacobi pela forma matrical:")
print("A matriz pode resolvida?\n")
TestarFazerJacobiM(M5, b5)

In [ ]:

M5 = np.array([[16,-4,4,12],
               [-4,2,-1,-7],
               [4,-1,26,13],
               [12,-7,13,25]])
b5 = np.array([2,2,-1,-2])
print("Metodo de Gauss-Seidel pela forma escalar:")
print("A matriz pode resolvida?\n")
TestarFazerGauss_SeidelE(M5, b5)

In [ ]:

M5 = np.array([[16,-4,4,12],
               [-4,2,-1,-7],
               [4,-1,26,13],
               [12,-7,13,25]])
b5 = np.array([2,2,-1,-2])


print("Metodo de Gauss-Seidel pela forma Matricial:")
print("A matriz pode resolvida?\n")
TestarFazerGauss_SeidelM(M5, b5)

2 - Use todos os métodos (Gauss, LU com pivotação, Choleski, Jacobi e Gauss-Seidel) e marquem o tempo com `%timeit -n1` para o seguinte sistema:

In [ ]:
MF = np.random.randint(1,20,(10000,10000))
bF = np.random.randint(-1000,1000,10000)
print(MF)

In [ ]:
#Alguns metodos nao irao funcionar, pois nao vao atender as condiçoes pra convergencia
# Exemplo disso eh os metodos iterativos Jacobi e Gauss-Seidel, pois a diagonal da matriz provalvelmente nao vai ser dominante
# e o raio espectral possivelmente nao seja < 1
eliminacaoGaussianaSimples(MF,bF)
print("Tempo da eliminaçãoGaussiana:", eliminacaoGaussianaSimples(MF,bF))
%timeit -n1 eliminacaoGaussianaSimples(MF,bF)

In [ ]:
print("Decomposição LU:")
print("Função de decomposição:")
%timeit -n1 decompoeLU(MF)
L, U = decompoeLU(MF)
print("Função Resolve:", resolveLU(L,U,bf))
%timeit -n1 resolveLU(L,U,bf)

In [ ]:
print("LU com pivotação parcial:")
%timeit -n1 LUparcial(MF)
L, U, P = LUparcial(MF)
print("Resolve LU parcial:", resolveLUpar(L,U,P,bf))
%timeit -n1 resolveLUpar(L,U,P,bf)

In [ ]:
print("Cholesky:")
print("Verifica se o sistema pode ser resolvido, senao for retorna -1:", resolveCholesky(MF,bf))
%timeit -n1 resolveCholesky(MF,bf)

In [5]:
print("Metodo de Jacobi e Gauss-Seidel pela forma escalar e matricial:\n")

print("A matriz pode resolvida?\n")
print("Não. Ele falhou em uma das condições de convergência.")

# e o raio espectral possivelmente nao seja < 1
#%timeit -n1 TestarFazerJacobiE(MF, bF)
#%timeit -n1 TestarFazerJacobiM(MF, bF)
#%timeit -n1 TestarFazerGauss_SeidelE(MF, bF)
#%timeit -n1 TestarFazerGauss_SeidelM(MF, bF)

Metodo de Jacobi e Gauss-Seidel pela forma escalar e matricial:

A matriz pode resolvida?

Não. Ele falhou em uma das condições de convergência.
